# Gridworld with First Visit

In [1]:
import numpy as np

# display output
from random import uniform
import random
import time
from IPython.display import display, clear_output

In [2]:
actions = [[-1, 0], [0, 1], [1, 0], [0, -1]] #up, right, down, left = (clockwise from up) 
action_count = len(actions) # total number of actions
gridSize = 5 # create a square grid of gridSize by gridSize
state_count = gridSize*gridSize # total number of states

In [3]:
class Gridworld():
    def __init__(self, gridSize):
        self.valueMap = np.zeros((gridSize, gridSize))
        self.states = [[i, j] for i in range(gridSize) for j in range(gridSize)]
        self.size = gridSize
        self.new_pos = [0, 0] # initialize new position for p_transition
        self.transition_prob = 1 # deterministic
    
    def initial_state(self):        # return initial state
        return grid.states[gridSize*gridSize-1]
   
    def transition_reward(self, current_pos, action): # return the transition probability

        # get next position: state: [0, 0], action: [0, 1], new_state = [0, 1]
        self.new_pos = np.array(current_pos) + np.array(action)

        # normally, reward = 0
        reward = 0

        # if new pos results in off the grid, return reward -1
        if -1 in self.new_pos or self.size in self.new_pos:
            reward = -1
        # if in state A, receive + 10
        if current_pos == [0, 1]:
            reward = 10
        # if in state B, receive + 5
        if current_pos == [0, 3]:
            reward = 5

        # if taking an action crosses the border; agent's new_pos is the same as the current pos
        if -1 in self.new_pos or self.size in self.new_pos: 
            self.new_pos = current_pos
            
        # if in state A, transition to state A'
        if current_pos == [0, 1]:
            self.new_pos = [4, 1]
            
        # if in state B, transition to state B'
        if current_pos == [0, 3]:
            self.new_pos = [2, 3]

        return self.new_pos, reward

## First-visit MC Control 

In [4]:
# create a grid object
grid = Gridworld(5)

In [5]:
# Initiate a random policy
random_policy = np.random.randint(1000, size=(state_count, action_count))
random_policy = random_policy/random_policy.sum(axis=1)[:,None]
policy = random_policy

In [6]:
# random policy
policy

array([[0.22573363, 0.42167043, 0.30474041, 0.04785553],
       [0.27693447, 0.36430951, 0.16734543, 0.19141059],
       [0.31578947, 0.24966262, 0.00179937, 0.43274854],
       [0.05868989, 0.35327527, 0.3517607 , 0.23627414],
       [0.25923416, 0.32226364, 0.19451034, 0.22399187],
       [0.33255597, 0.30597015, 0.29197761, 0.06949627],
       [0.17191283, 0.12881356, 0.24939467, 0.44987893],
       [0.22834646, 0.4090344 , 0.1931206 , 0.16949855],
       [0.06806283, 0.32518906, 0.53810355, 0.06864456],
       [0.01007109, 0.55509479, 0.33530806, 0.09952607],
       [0.3063263 , 0.12874584, 0.41176471, 0.15316315],
       [0.05776173, 0.11642599, 0.82490975, 0.00090253],
       [0.08752026, 0.35872501, 0.32793085, 0.22582388],
       [0.30947776, 0.17359768, 0.10444874, 0.41247582],
       [0.04180887, 0.12372014, 0.63737201, 0.19709898],
       [0.1145    , 0.102     , 0.3675    , 0.416     ],
       [0.23745174, 0.39720077, 0.32094595, 0.04440154],
       [0.31547619, 0.01360544,

### Create an Episode following Policy

In [7]:
def generate_episode(steps):

    # set initial state
    state_vector = grid.initial_state()

    # initialize state (with iniitial state), action list and reward list
    state_list = [state_vector]
    action_list = []
    reward_list = []

    # generate an episode
    for i in range(steps):

        # pick an action based on categorical distribution in policy
        action_index = int(np.random.choice(action_count, 1, p=policy[grid.states.index(state_vector)])) 
        action_vector = actions[action_index] # convert the integer index (ie. 0) to action (ie. [-1, 0])

        # get new state and reward after taking action from current state
        new_state_vector, reward = grid.transition_reward(state_vector, action_vector)
        state_vector = list(new_state_vector)

        # save state, action chosen and reward to list
        state_list.append(state_vector)
        action_list.append(action_vector)
        reward_list.append(reward)
        
    return state_list, action_list, reward_list

### First Visit MC

In [8]:
# initialize q values for all state action pairs
Q_values = np.zeros((state_count, action_count))

In [9]:
# intialize parameters
gamma = 0.99
epsilon = 0.2

In [10]:
# define average function
def Average(lst): 
    return sum(lst) / len(lst) 

In [11]:
# iterate 500 times: each time, generating an episode of 200 steps
max_steps = 200

# define variables for keeping track of time steps
Terminal = max_steps
t_list=[]
for i in range(1,max_steps+1):
    t = Terminal - i
    t_list.append(t)

In [12]:
# iteration 500 times
for iteration in range(500):
  
    # generate an episode of specified step count
    state_list, action_list, reward_list = generate_episode(max_steps)
    
#     print("state_list: ", state_list)
#     print("action_list: ", action_list)
#     print("reward_list: ", reward_list)
    
    # intialize G
    G = 0
    
    # initiate returns and visited list to none
    returns_list = []
    visited_list = []

    # loop for each step of episode: T-1, T-2, T-3 ... 0 = 199, 198, 197 ... 0
    for t in t_list:

        # calculate G: starting with the last reward at index t (naturally accounts for pseudocode's "t-1")
        G = gamma*G + reward_list[t]
        # print(G)
        
        # combine state action pair, for example, state = [0,0], action = [0,1], state_action_pair = [0,0,0,1]
        state_action_pair = []
        state_action_pair.extend(state_list[t])
        state_action_pair.extend(action_list[t])
        # print(state_action_pair)

        # check if state action pair have been visited before (if not: continue, else: move to the next time step)
        if state_action_pair not in visited_list:

            # add state action pair to visited list
            visited_list.append(state_action_pair)

            # append G to returns
            returns_list.append(G)

            # find state and action index, for example, converting action [-1, 0] to 0, and same for state #
            state_index = grid.states.index(state_list[t])
            action_index = actions.index(action_list[t])
    #         print("state_index: ", state_index)

            # write Q_values to the state-action pair
            Q_values[state_index][action_index] = Average(returns_list)
    #         print("average return: ", Average(returns_list))

            # choose best action at given state
            choose_action = np.argmax(Q_values[state_index])

            # overwrite policy
            for a in range(action_count): # for action in actions [0, 1, 2, 3]
                if choose_action == a: # if the choose_action is the same as the current action
                    policy[state_index][a] = 1 - epsilon + epsilon/action_count 
                else: # if choose_action is not the same as the current action
                    policy[state_index][a] = epsilon/action_count

In [13]:
# total unique state action pairs at the end of one episode
print(len(visited_list))

41


In [14]:
np.set_printoptions(precision=2)
Q_values

array([[-19.32, -20.85,  -6.11,  -2.67],
       [-20.98, -16.55, -15.07,   3.33],
       [ -0.77,  -0.79,  -9.83, -16.36],
       [-15.54,   8.58, -16.64, -19.13],
       [  1.4 ,   1.53,   9.58,   8.95],
       [-20.73, -12.39, -17.5 , -27.87],
       [  4.97,   9.75,   6.57, -11.23],
       [-17.39,   9.81, -19.51, -30.04],
       [-19.3 ,   9.74, -16.72, -18.23],
       [  9.28,  -3.27,   9.87,   9.87],
       [-20.6 ,   6.99, -18.18, -18.91],
       [  5.94,   7.96,   7.5 ,   7.29],
       [-31.31, -19.42,  -1.71,   7.71],
       [-18.92,   8.8 ,  -0.53,   8.17],
       [  9.91,   9.91,   9.02,  -6.37],
       [  7.45,  -0.33,   7.7 ,  -1.98],
       [  7.42,   7.73,   0.  ,   7.56],
       [-32.37, -16.54,   7.73, -20.03],
       [ -0.57, -34.64, -35.64, -33.26],
       [-36.81,   9.19,   9.32, -34.01],
       [ -0.42,   7.64,  -0.76,  -1.  ],
       [  0.  ,  -0.58,   7.65,   7.58],
       [ -9.3 , -20.49,  -1.3 ,   7.72],
       [-35.18,   9.42, -39.4 ,   9.64],
       [-37.  , 

In [15]:
#up, right, down, left = (clockwise from up) 
policy

array([[0.05, 0.05, 0.05, 0.85],
       [0.05, 0.05, 0.05, 0.85],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.85, 0.05, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.85, 0.05, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.85, 0.05, 0.05],
       [0.05, 0.05, 0.85, 0.05],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.05, 0.05, 0.85],
       [0.05, 0.05, 0.05, 0.85]])

## Visualize 

In [16]:
# PRINT POLICY TABLE ################################################################################
# import pandas library
import pandas as pd
# define column and index
columns=range(grid.size)
index = range(grid.size)
# define dataframe to represent policy table
policy_table = pd.DataFrame(index = index, columns=columns)

# iterate through policy to make a table that represents action number
# as action name (eg. left, right, up, down)
for state in range(len(policy)):
    
    # find the best action at each state
    best_action = np.argmax(policy[state])

    # get action name
    if best_action == 0:
        action_name = 'up'
    elif best_action == 1:
        action_name = 'right'
    elif best_action == 2:
        action_name = 'down'
    else:
        action_name = 'left'

    # calculate the row and column coordinate of the current state number
    row = int(state/grid.size)
    column = round((state/grid.size - int(state/grid.size))*grid.size)
            
    # assign action name
    policy_table.loc[row][column] = action_name

print("Policy Table: ")
print(policy_table)
print()

Policy Table: 
       0      1      2      3     4
0   left   left     up  right  down
1  right  right  right  right  left
2  right  right   left  right    up
3   down  right   down     up  down
4  right   down   left   left  left

